In [1]:
import os
import re

import numpy as np
import pandas as pd

# os.chdir("/home/yujia/Project/2023-07-20-individual_MR/dat/02_base_data/")

In [2]:
LDL = pd.read_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/download/without_UKB_LDL_INV_AFR_HRC_1KGP3_others_ALL.meta.singlevar.results.gz", sep="\t", compression = 'gzip')

print("Total SNP in the GWAS data: ")
print(LDL.shape)

Total SNP in the GWAS data: 
(31627319, 14)


In [3]:
LDL

,rsID,CHROM,POS_b37,REF,ALT,N,N_studies,POOLED_ALT_AF,EFFECT_SIZE,SE,pvalue_neg_log10,pvalue,pvalue_neg_log10_GC,pvalue_GC
0,rs376342519,1,10616,C,CCGCCGTTGCAAAGGCGCGCCG,3818,2,0.02440,0.394101,0.180654,1.535440,0.0291,1.478144,0.0333
1,rs575272151,1,11008,C,G,3818,2,0.04900,-0.032971,0.047118,0.315082,0.4840,0.305686,0.4950
2,rs544419019,1,11012,C,G,3818,2,0.04900,-0.032971,0.047118,0.315082,0.4840,0.305686,0.4950
3,rs62635286,1,13116,T,G,3818,2,0.01490,-0.133677,0.073616,1.158695,0.0694,1.117067,0.0764
4,rs62028691,1,13118,A,G,3818,2,0.01490,-0.133677,0.073616,1.158695,0.0694,1.117067,0.0764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31627314,rs8142977,22,51239304,C,T,3818,2,0.00602,0.101846,0.107283,0.465393,0.3420,0.450744,0.3540
31627315,rs561893765,22,51239794,C,A,18023,7,0.03040,-0.001391,0.031133,0.015762,0.9640,0.015376,0.9650
31627316,rs5770837,22,51240820,C,T,5506,3,0.11300,0.036446,0.033678,0.554143,0.2790,0.536252,0.2910
31627317,rs7287738,22,51241285,T,G,6399,2,0.00484,-0.008761,0.112746,0.027769,0.9380,0.027080,0.9400


In [4]:
# rename the data
LDL.rename(columns={"CHROM":"CHR", "POS_b37":"BP", "rsID":"SNP", "REF":"A2", "ALT":"A1", "POOLED_ALT_AF":"EAF", "EFFECT_SIZE":"BETA", "SE":"SE", "pvalue":"P", "N":"N"}, inplace=True)

# convert EAF to MAF
LDL['MAF'] = LDL['EAF'].apply(lambda x : x if x <= 0.5 else 1-x)

# Uppercase the A1 and A2
LDL['A1'] = LDL['A1'].str.upper()
LDL['A2'] = LDL['A2'].str.upper()

In [5]:
LDL

,SNP,CHR,BP,A2,A1,N,N_studies,EAF,BETA,SE,pvalue_neg_log10,P,pvalue_neg_log10_GC,pvalue_GC,MAF
0,rs376342519,1,10616,C,CCGCCGTTGCAAAGGCGCGCCG,3818,2,0.02440,0.394101,0.180654,1.535440,0.0291,1.478144,0.0333,0.02440
1,rs575272151,1,11008,C,G,3818,2,0.04900,-0.032971,0.047118,0.315082,0.4840,0.305686,0.4950,0.04900
2,rs544419019,1,11012,C,G,3818,2,0.04900,-0.032971,0.047118,0.315082,0.4840,0.305686,0.4950,0.04900
3,rs62635286,1,13116,T,G,3818,2,0.01490,-0.133677,0.073616,1.158695,0.0694,1.117067,0.0764,0.01490
4,rs62028691,1,13118,A,G,3818,2,0.01490,-0.133677,0.073616,1.158695,0.0694,1.117067,0.0764,0.01490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31627314,rs8142977,22,51239304,C,T,3818,2,0.00602,0.101846,0.107283,0.465393,0.3420,0.450744,0.3540,0.00602
31627315,rs561893765,22,51239794,C,A,18023,7,0.03040,-0.001391,0.031133,0.015762,0.9640,0.015376,0.9650,0.03040
31627316,rs5770837,22,51240820,C,T,5506,3,0.11300,0.036446,0.033678,0.554143,0.2790,0.536252,0.2910,0.11300
31627317,rs7287738,22,51241285,T,G,6399,2,0.00484,-0.008761,0.112746,0.027769,0.9380,0.027080,0.9400,0.00484


In [6]:
# reorder the column
cols = ["CHR", "BP", "SNP", "A1", "A2", "N", "SE", "P", "BETA", "MAF", "EAF"]
LDL = LDL[cols]

# Remove NaN row
LDL = LDL.dropna()

# convert to correct column type
LDL = LDL.astype({'CHR':'int', 'BP':'int', "SNP":'str', 'A1':'str', 'A2':'str', 'N':'int', 'SE':'float', 'P':'float', 'BETA':'float', 'MAF':'float', 'EAF':'float'})

# remove SNP without SNPID
LDL = LDL[LDL['SNP'].str.startswith('rs')]

# save the formatted results
LDL.to_csv("/mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL_formated_nadrop.tsv.gz", sep="\t", index=False)

print("SNP left after basic QC: ")
print(LDL.shape)

SNP left after basic QC: 
(31481851, 11)


In [7]:
LDL

,CHR,BP,SNP,A1,A2,N,SE,P,BETA,MAF,EAF
0,1,10616,rs376342519,CCGCCGTTGCAAAGGCGCGCCG,C,3818,0.180654,0.0291,0.394101,0.02440,0.02440
1,1,11008,rs575272151,G,C,3818,0.047118,0.4840,-0.032971,0.04900,0.04900
2,1,11012,rs544419019,G,C,3818,0.047118,0.4840,-0.032971,0.04900,0.04900
3,1,13116,rs62635286,G,T,3818,0.073616,0.0694,-0.133677,0.01490,0.01490
4,1,13118,rs62028691,G,A,3818,0.073616,0.0694,-0.133677,0.01490,0.01490
...,...,...,...,...,...,...,...,...,...,...,...
31627314,22,51239304,rs8142977,T,C,3818,0.107283,0.3420,0.101846,0.00602,0.00602
31627315,22,51239794,rs561893765,A,C,18023,0.031133,0.9640,-0.001391,0.03040,0.03040
31627316,22,51240820,rs5770837,T,C,5506,0.033678,0.2790,0.036446,0.11300,0.11300
31627317,22,51241285,rs7287738,G,T,6399,0.112746,0.9380,-0.008761,0.00484,0.00484


In [8]:
# =============================
# Step 2: remove SNPs with MAF < 0.01
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL_formated_nadrop.tsv.gz |" + \
      "awk 'NR==1 || ($10 > 0.01) {print}' |" + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.standardGWASQC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.standardGWASQC.gz | wc -l"
print("SNP left after removing SNPs with MAF < 0.01: ")
print(os.popen(cmd).read())

SNP left after removing SNPs with MAF < 0.01: 
16264187



In [9]:
# =============================
# Step 3: remove duplicate SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.standardGWASQC.gz |" + \
      "awk '{seen[$3]++; if(seen[$3]==1){ print}}' |" + \
      "gzip - > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.nodup.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.nodup.gz | wc -l"
print("SNP left after removing duplicate SNPs: ")
print(os.popen(cmd).read())

SNP left after removing duplicate SNPs: 
16212920



In [10]:
# =============================
# Step 4: keep non-ambiguous SNPS
# =============================

cmd = "gunzip -c /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.nodup.gz | " + \
      "awk '!( ($4==\"A\" && $5==\"T\") || ($4==\"T\" && $5==\"A\") || ($4==\"G\" && $5==\"C\") || ($4==\"C\" && $5==\"G\")) {print}' | " + \
      "gzip > /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.QC.gz"
os.system(cmd)

cmd = "zcat /mnt/md0/yujia/project/2023-07-20-individual_MR/dat/02_base_data/LDL/QC_AFR/LDL.QC.gz | wc -l"
print("SNP left after removing ambiguous SNPs: ")
print(os.popen(cmd).read())

SNP left after removing ambiguous SNPs: 
13919340

